In [1]:
#import the libraries
import os
import time
import requests
import numpy as np
import pandas as pd
import math
import uuid
from bs4 import BeautifulSoup

def review_scraper(url):
    hdr = {'User-Agent': 'Chrome/39.0.2171.71'}
    response = requests.get(url,headers=hdr)
    soup = BeautifulSoup(response.content, "html.parser")
    Date_n_JobTitle=[]
    Date=[]
    JobTitle=[]
    AuthorLocation=[]
    OverallRating=[]
    Pros=[]
    Cons=[]  

    #get the Posted Date and Job Title
    for x in soup.find_all('span', {'class':'authorJobTitle middle common__EiReviewDetailsStyle__newGrey'}):
        Date_n_JobTitle.append(x.text)

    #get the Posted Date
    for x in Date_n_JobTitle:
        Date.append(x.split(' -')[0])

    #get Job Title
    for x in Date_n_JobTitle:
        JobTitle.append(x.split(' -')[1])

    #get Author Location
    for x in soup.find_all('span', {'class':'authorLocation'}):
        AuthorLocation.append(x.text)

    #get Overall Rating
    for x in soup.find_all('span', {'class':'ratingNumber mr-xsm'}):
        OverallRating.append(float(x.text))

    #get Pros
    for x in soup.find_all('span', {'data-test':'pros'}):
        Pros.append(x.text)

    #get Cons
    for x in soup.find_all('span', {'data-test':'cons'}):
        Cons.append(x.text)

    #putting everything together
    Reviews = pd.DataFrame(list(zip( Date, JobTitle, AuthorLocation, OverallRating, Pros, Cons)), 
                    columns = [ 'createdDate', 'userDesignation', 'userLocation', 'OverallRating', 'Pros', 'Cons'])

    return Reviews   
#paste/replace the url to the first page of thehttps company's Glassdoor review in between the ""
input_url="https://www.glassdoor.co.in/Reviews/Atrium-Reviews-E1792887" 
reviews_df = []
reviews_df = review_scraper(input_url+".htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME")
for x in range(2,50):
    url = input_url+"_P"+str(x)+".htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng"
    reviews_df = reviews_df.append(review_scraper(url), ignore_index=True)
reviews_df['orgName'] = 'atrium' #add orgName

In [2]:
reviews_df

,createdDate,userDesignation,userLocation,OverallRating,Pros,Cons,orgName
0,"Oct 11, 2022",Interior Design In Charge,Dubai,4.0,"Friendly working environment , all employees w...",too much closeness to every workers,atrium
1,"Oct 10, 2022",Account Executive,"San Francisco, CA",3.0,Most people at the company are nice to work with.,Upper management is manipulative. Customers ch...,atrium
2,"Sep 15, 2022",Business Operations Associate,"San Francisco, CA",5.0,We were doing a super cool mission to make leg...,"Company shut down, which impacted a lot of peo...",atrium
3,"Jun 13, 2022",Corporate Paralegal,"Lexington, NC",4.0,Great people and company culture,"Chaos, lack of organization, distractions",atrium
4,"May 17, 2021",Attorney,"San Francisco, CA",3.0,Colleagues were really smart and collaborative.,"The firm went under, I believe, because the ce...",atrium
5,"Apr 16, 2021",Junior Architect,"Moscow, Moskva",4.0,"Friendly atmosphere, opportunities to learn ne...",Lack of benefits and promotion mechanisms,atrium
6,"Mar 19, 2021",Practice Assistant,"San Francisco, CA",3.0,Free snacks and tons of perks,Too much change too often,atrium
7,"Feb 3, 2021",People Team,"San Francisco, CA",5.0,fantastic team\r\nflexible environment\r\nperk...,closing the doors after so much great work was...,atrium
8,"Jan 29, 2021",Senior Corporate Paralegal,"San Francisco, CA",4.0,"Loved the start-up culture, collaborative envi...",Company grew too fast and spent too much money...,atrium
9,"Jun 18, 2020",Counselor,"Charlotte, NC",4.0,Atrium has many good people.,Sometimes there are problems with staff commun...,atrium


In [3]:
reviews_df.to_excel('atrium.xlsx', sheet_name='Sheet1', index=False) #convert dataframe to xlsx format